<a href="https://colab.research.google.com/github/Jenni01-12/Parcial-Final/blob/main/Copia_de_Parcial_final_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalación de librerías**

In [ ]:
# --- 0. Configuración Inicial ---
# Nombre del modelo Ollama a utilizar (cambia esto para usar un modelo diferente)
# Asegúrate de que este modelo exista en Ollama y sea adecuado para tareas de generación de código.
# Modelos como 'codellama', 'deepseek-coder', 'llama3' suelen ser buenos para esto.
# 'llama3' es una buena opción general.
ollama_model_name = "deepseek-r1" # Prueba con llama3, o "codellama" si quieres enfocarte en código
# ollama_model_name = "llama2:7b" # Puedes volver a llama2 si prefieres

# Nombre del archivo CSV si decides usar uno real (deja None para usar datos simulados)
# Para usar un archivo real:
# 1. Sube tu archivo CSV a tu sesión de Colab.
# 2. Cambia esta variable a "nombre_de_tu_archivo.csv"
csv_file_name = "Gravedad.csv" # Cambia a "tu_archivo.csv" para usar un archivo real

# --- 1. Instalación de Librerías y Ollama ---

print("--- Instalando librerías Python ---")
# Instalamos/actualizamos pandas, numpy, ollama y gradio
# Usamos --upgrade para asegurar versiones compatibles y -q para reducir el output
# ¡pandas y numpy ya deberían estar bien de la vez anterior, pero --upgrade no hace daño!
!pip install --upgrade pandas numpy ollama gradio openpyxl -q

# **Verificación de importaciones**
print("\n--- Verificando importaciones básicas ---")
try:
    import pandas as pd
    import numpy as np
    import ollama
    import gradio as gr
    import matplotlib.pyplot as plt # Para plots generados por código
    import io
    import sys # Importar sys para capturar stdout
    print("¡pandas, numpy, ollama, gradio, matplotlib, io y sys importados correctamente!")
except ImportError as e:
    print(f"ERROR CRÍTICO: No se pudo importar una librería necesaria: {e}")
    print("Asegúrate de que las instalaciones se completaron sin errores graves.")
    # import sys; sys.exit("Fallo en importaciones básicas.") # Descomentar si quieres detenerte aquí
except Exception as e:
    print(f"ERROR inesperado durante la importación: {e}")
    # import sys; sys.exit("Fallo en importaciones básicas.") # Descomentar si quieres detenerte aquí


# Instalar Ollama en Colab (solo es necesario ejecutar una vez por sesión)
print("\n--- Instalando Ollama ---")
!curl -fsSL https://ollama.com/install.sh | sh

# --- 2. Iniciar el servidor de Ollama en segundo plano ---
print("\n--- Iniciando servidor Ollama ---")
!pkill ollama || true # Asegura que no hay procesos ollama corriendo
!nohup /usr/local/bin/ollama serve > ollama_output.log 2>&1 & # Ejecuta en segundo plano

# --- 3. Esperar a que el servidor de Ollama inicie completamente ---
import time
print("Esperando a que el servidor Ollama esté listo (15 segundos)...")
time.sleep(15) # Ajusta este tiempo si ves errores de conexión iniciales

# --- 4. Verificar que el servidor esté respondiendo ---
print("\n--- Verificando servidor Ollama ---")
!curl -s http://localhost:11434/api/tags || echo "El servidor Ollama no está respondiendo. Revisa los logs o aumenta el tiempo de espera."

# --- 5. Descargar el modelo especificado ---
print(f"\n--- Descargando modelo {ollama_model_name} desde Ollama ---")
# Si cambiaste ollama_model_name, esto descargará el nuevo. Si ya lo tienes, será rápido.
!ollama pull {ollama_model_name}
print(f"Modelo {ollama_model_name} descargado (o ya disponible).")

# --- 6. Preparar los datos (Simulados o cargados desde CSV) ---
# pandas y os ya deberían estar importados desde la verificación inicial
import os

print("\n--- Cargando datos ---")

df = None # Inicializar el dataframe a None

if csv_file_name and os.path.exists(csv_file_name):
    # Si se especificó un archivo CSV y existe, cargarlo
    try:
        df = pd.read_csv(csv_file_name)
        print(f"Datos cargados desde '{csv_file_name}'. Filas: {len(df)}, Columnas: {len(df.columns)}")
    except Exception as e:
        print(f"Error al cargar el archivo CSV '{csv_file_name}': {e}")
        print("Se procederá a usar datos simulados.")
        csv_file_name = None # Revertir para usar datos simulados si falla la carga
else:
    if csv_file_name:
        print(f"Archivo '{csv_file_name}' no encontrado.")
        print("Se procederá a usar datos simulados.")
        csv_file_name = None # Revertir para usar datos simulados si no se encuentra


--- Instalando librerías Python ---

--- Verificando importaciones básicas ---
¡pandas, numpy, ollama, gradio, matplotlib, io y sys importados correctamente!

--- Instalando Ollama ---
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
################                                                          23.3%

In [ ]:
print("\nPrimeras 5 filas del DataFrame:")
print(df.head().to_markdown(index=False)) # Mostrar como markdown para mejor legibilidad en Colab
print(f"\nColumnas y tipos de datos:")
display(df) # Usar df.info() directamente para mostrar en la consola de Colab
# --- 7. Funciones para interactuar con Ollama y ejecutar código ---

# Pre-generar la descripción del DataFrame que se enviará al LLM
# Esto se hace una vez después de cargar o crear el DataFrame
# Usamos io.StringIO() para capturar el output de df.info() como string para el LLM
buffer = io.StringIO()
df.info(verbose=True, buf=buffer)
dataframe_description = f"""Tienes acceso a un pandas DataFrame llamado 'df'.
Su información (columnas, tipos, no-nulos) es:
{buffer.getvalue()}

Primeras filas:
{df.head().to_string()}
"""


In [ ]:
def generate_python_code(prompt, dataframe_info=dataframe_description, model=ollama_model_name):
    """
    Genera código Python ejecutable sin secciones de razonamiento
    """
    system_prompt = f"""Eres un generador de código Python especializado en análisis económico.
    Reglas estrictas:
    1. Devuelve SOLO código Python ejecutable (sin comentarios tipo <think>)
    2. El código debe:
       - Usar el DataFrame 'df'
       - Crear gráficos con matplotlib/seaborn
       - Guardar la figura en variable 'fig'
    3. Estructura obligatoria:
       - Importar librerías necesarias
       - Procesamiento de datos
       - Visualización
       - Retorno de figura (fig = plt.gcf())
    4. Prohibido:
       - Secciones de razonamiento
       - Comentarios extensos
       - Bloques markdown
    Datos disponibles: {dataframe_info}
    """

    user_prompt = f"Pregunta del usuario: {prompt}"

    print(f"\n--- Enviando prompt a {model} para generar código ---")
    # Usamos la librería ollama para interactuar con el servidor local
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_prompt},
            ],
            options={
                "temperature": 0.2, # Mantener bajo para código preciso
                # "num_predict": 512 # Opcional: limitar longitud si es necesario
            }
        )
        # Asegurarse de que solo se extrae el contenido de la respuesta del modelo
        code = response['message']['content'].strip()

        # A veces, los LLMs pueden incluir markdown a pesar de las instrucciones estrictas.
        # Intentar limpiar bloques de código markdown si aparecen.
        if code.startswith("```python"):
            code = code.replace("```python", "").strip()
            if code.endswith("```"):
                code = code[:-3].strip()
        elif code.startswith("```"): # Otros bloques de código
             code = code.replace("```", "").strip()


        print("Código Python generado:")
        print("--- INICIO CÓDIGO ---")
        print(code)
        print("--- FIN CÓDIGO ---")
        return code
    except Exception as e:
        print(f"Error al llamar a Ollama para generar código: {e}")
        return None


In [ ]:
def execute_python_code(code, dataframe):
    """
    Ejecuta código Python con validaciones mejoradas y manejo de errores robusto.
    Devuelve:
    - Para gráficos: objeto matplotlib.Figure
    - Para texto: str con output
    - Para errores: str descriptivo
    """
    # --- Validaciones iniciales ---
    if "<think>" in code or "```" in code:
        return "Error: El código contiene secciones no ejecutables (remove <think> o markdown)"

    required_vars = {'pd', 'plt', 'df'}
    if not all(var in code for var in required_vars):
        return f"Error: Faltan imports esenciales. Requeridos: {required_vars}"
    print("\n--- Ejecutando código Python ---")
    # Usamos exec para ejecutar código en un entorno controlado
    # Creamos un diccionario para el entorno de ejecución, incluyendo el DataFrame 'df' y librerías comunes
    execution_env = {'df': dataframe, 'pd': pd, 'np': np, 'plt': plt} # Usamos 'np' y 'plt'

    # Redirigir stdout para capturar output de print
    old_stdout = sys.stdout
    redirected_output = io.StringIO()
    sys.stdout = redirected_output

    # Usamos un diccionario para capturar el resultado si el código lo asigna a una variable 'result'
    # o si la última expresión no es print.
    result_holder = {'_result': None}
    code_to_exec = f"import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n{code}\n\n_result = None\ntry:\n    _result = eval(compile(code_to_exec, '<string>', 'eval'), execution_env)\nexcept:\n    pass # Could not evaluate as single expression"
     # ^^ Intentamos envolverlo para capturar la última expresión, pero esto puede ser frágil.
     # Un enfoque más robusto es confiar en que el LLM use print() o retorne una figura.

    # Simplificamos la ejecución para confiar más en el `print` del código generado o en el retorno de la figura
    try:
        # Ejecutar el código en el entorno preparado
        exec(code, execution_env)

        # Restaurar stdout
        sys.stdout = old_stdout
        output_text = redirected_output.getvalue().strip() # Obtener output impreso

        # Intentar detectar si se generó un plot
        fig = None
        try:
            # Si hay figuras activas y no se cerraron explícitamente
            # get_fignums() devuelve una lista de ids de figuras activas
            if plt.get_fignums():
                 # Asumimos que la última figura creada es la relevante
                 fig = plt.gcf() # Get Current Figure
                 print("(Código ejecutado, se detectó una figura de matplotlib)") # Mensaje para el log
                 # Es buena práctica limpiar las figuras después de obtener la que queremos
                 # plt.close('all') # Podríamos cerrarlas aquí, o después de retornarlas en el handler de Gradio
        except Exception as fig_e:
             print(f"Advertencia al verificar figuras de matplotlib: {fig_e}")
             pass # No fallar si falla la detección de figura

        # Determinar el resultado a devolver
        if fig:
            # Si hay una figura, devolver la figura (Gradio la mostrará en el componente Plot)
            return fig
        elif output_text:
            # Si hay output de texto, devolverlo (Gradio lo mostrará en el componente Textbox)
            return output_text
        else:
            # Si no hay ni figura ni output de texto
            # Podríamos intentar devolver el valor de result_holder['_result'] si lo implementamos,
            # pero basándonos en las instrucciones del prompt, el LLM debería imprimir o generar figura.
            # Retornar un mensaje informativo.
            return "Código ejecutado. No se produjo output de texto ni figura visible. Revisa el prompt o el código generado."

    except Exception as e:
        # Restaurar stdout si ocurrió un error antes de restaurarlo
        sys.stdout = old_stdout
        # Limpiar figuras si hubo un error para evitar que interfieran en la siguiente ejecución
        plt.close('all')
        print(f"Error al ejecutar el código: {e}")
        # Devolver el mensaje de error a la interfaz
        # Incluimos el código ejecutado para depuración en la interfaz
        return f"ERROR al ejecutar el código:\n{e}\n\nCódigo ejecutado:\n```python\n{code}\n```"

In [ ]:
def analyze_generated_plot(figure, prompt_context, model=ollama_model_name):
    """Analiza un gráfico matplotlib ya generado y devuelve insights"""
    import tempfile
    import base64
    import os

    # Guardar el gráfico temporalmente
    temp_img = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
    figure.savefig(temp_img.name, bbox_inches='tight')
    temp_img.close()

    # Leer la imagen en base64
    with open(temp_img.name, 'rb') as img_file:
        encoded_img = base64.b64encode(img_file.read()).decode('utf-8')

    system_prompt = """Eres un intérprete visual de datos económicos. Analiza el gráfico proporcionado y:
    1. Identifica patrones clave (máx. 3)
    2. Explica su relevancia económica
    3. Destaca anomalías (si existen)

    Formato de respuesta:
    - Patrón 1: [descripción] → [significado económico]
    - Patrón 2: [descripción] → [significado económico]
    - Anomalía: [descripción] → [posible causa]
    """

    try:
        response = ollama.chat(
            model=model,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': [
                    {'type': 'text', 'text': f"Contexto: {prompt_context}"},
                    {'type': 'image_url', 'image_url': f"data:image/png;base64,{encoded_img}"}
                ]}
            ]
        )
        return response['message']['content']
    except Exception as e:
        print(f"Error analizando gráfico: {e}")
        return None
    finally:
        os.unlink(temp_img.name)

def generate_markdown_summary(plot_analysis, prompt_context):
    """Genera resumen para PPT en markdown basado en el análisis"""
    system_prompt = """Genera un resumen ejecutivo en markdown para PowerPoint con:
    # Título
    - **Hallazgos**: 3 bullet points
    - **Conclusión**: 1 frase
    - **Recomendación**: 1 acción
    """
    try:
        response = ollama.chat(
            model=ollama_model_name,
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': f"Análisis: {plot_analysis}\nContexto: {prompt_context}"}
            ]
        )
        return response['message']['content']
    except Exception as e:
        print(f"Error generando resumen: {e}")
        return "## Error\nNo se pudo generar el resumen"

In [ ]:
# Función que Gradio llamará
def chatbot_response(user_input):
    """Procesa la entrada del usuario, genera y ejecuta código, y devuelve: texto, gráfico y resumen markdown"""
    global df # Asegúrate de que df está accesible (definido en la Celda 6)

    if df is None:
        return "Error: DataFrame no cargado. Verifica la configuración.", None, None # Retornar 3 valores (texto, None)

    # Opciones especiales para la interfaz (opcional, puedes quitar si no las necesitas)
    if user_input.lower() == 'mostrar dataframe info':
        buffer = io.StringIO()
        df.info(verbose=True, buf=buffer)
        return buffer.getvalue(), None, None
    if user_input.lower() == 'mostrar primeras filas':
        return df.head().to_markdown(index=False), None, None
    if user_input.lower() == 'mostrar datos simulados':
        return df.to_markdown(index=False), None, None


    # Generar el código Python usando Ollama
    code_to_execute = generate_python_code(user_input, dataframe_description, model=ollama_model_name)

    # Limpiar figuras existentes antes de ejecutar un nuevo código que podría crear plots
    plt.close('all')

    if code_to_execute:
        # Ejecutar el código generado
        execution_result = execute_python_code(code_to_execute, df)

    #creamos las funciones
    markdown_summary = None
    analysis_text = None

        # Gradio puede manejar la distinción entre texto y figura si la función retorna el tipo correcto
        # Si execution_result es un string, Gradio lo pondrá en el primer output (Textbox)
        # Si execution_result es un Figure, Gradio lo pondrá en el segundo output (Plot)
        # Debemos devolver una tupla (texto, plot) donde uno es el resultado y el otro es None
    if isinstance(execution_result, plt.Figure):
        # 1. Análisis del gráfico (nuevo)
        plot_analysis = analyze_generated_plot(execution_result, user_input)

        if plot_analysis:
            analysis_text = f"**Análisis del gráfico**:\n{plot_analysis}"
            # 2. Generar resumen markdown (nuevo)
            markdown_summary = generate_markdown_summary(plot_analysis, user_input)

        return analysis_text, execution_result, markdown_summary

    elif isinstance(execution_result, str):
        return execution_result, None, None  # Mantiene output original + markdown None

    else:
        return f"Resultado inesperado: {type(execution_result)}", None, None


print("\n--- Configurando interfaz Gradio ---")

# Crear la interfaz Gradio
if df is not None: # Solo si el DataFrame se cargó correctamente
    interface = gr.Interface(
        fn=chatbot_response, # La función a llamar
        inputs=gr.Textbox(label="Tu pregunta sobre los datos o comando especial"), # Un campo de texto para la entrada
        outputs=[
            gr.Textbox(label="Análisis", interactive=False, visible=True), # Campo para respuestas de texto, siempre visible
            gr.Plot(label="Gráfico", visible=True), # Campo para gráficos, siempre visible
            gr.Markdown(label="Resumen PPT", visible=True)  # Aseguramos que sea visible
            # Gradio ocultará automáticamente los componentes si el valor retornado es None
        ],
        title="CSV Chatbot Mejorado",
        description=f"""Ahora con análisis automático y resumen para presentaciones.<br>
        Columnas: {', '.join(df.columns)}. Modelo: {ollama_model_name}<br>
        Comandos especiales: `mostrar dataframe info`, `mostrar primeras filas`""",
        live=False,
        allow_flagging="never"
    )

    print("\n--- Iniciando interfaz Gradio ---")
    # share=True genera un enlace público de Hugging Face temporal.
    # debug=True muestra logs de Gradio (útil para depurar).
    # El puerto 7860 es el predeterminado de Gradio, asegúrate de que no esté en uso.
    # Usamos `inline=False` para forzar que el enlace se muestre claramente, no incrustado.
    interface.launch(share=True, debug=True, inline=False)

    print("\n--- Interfaz Gradio iniciada. Busca el enlace 'Public URL' en el output de la celda. ---")
else:
    print("\nDataFrame no cargado. No se puede iniciar la interfaz Gradio.")



In [ ]:
# 1. Configuración inicial
!pip install -q gradio ollama pandas matplotlib

# 2. Inicia Ollama (si usas modelo local)
!ollama serve &  # El '&' lo ejecuta en segundo plano

# 3. Lanza la interfaz
interface = gr.Interface(your_function, inputs, outputs)
interface.launch(share=True)  # share=True es clave para enlace público